# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} x_1 ~~ s.t. ~ x_1 + x_2 + x_3 = 1 ~, ~~ x \ge 0$

Dual: $\underset{\lambda \in \mathbb{R}, s \in \mathbb{R^3}}{\max} \lambda ~~ s.t. ~ [1~1~1]' \lambda + s = [1~0~0]'~, ~~s\ge0$

### 2)

In [50]:
using LinearAlgebra
using LaTeXStrings
using DataStructures

In [160]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end


display_results (generic function with 1 method)

In [179]:
function primal_dual_diff(c, b, x, lambda)
    return abs(c'*x - b'*lambda) / (1 + abs(c'*x))
end

function Point_in_Neighborhood_infty(gamma, beta, A, b, x, lambda, s, mu, x_0, lambda_0, s_0, mu_0)
    rb = A*x - b
    rb0 = A*x_0 - b
    rc = A'*lambda + s - c
    rc0 = A'*lambda_0 + s_0 - c
    xs_pv = x .* s
    
    # tester conditions sur norm([rb, rc]) ou sur norm(rb) et norm(rc indépendemment) ?
    return sum(x.<=0)==false && sum(s.<=0)==false && sum(xs_pv.<=gamma*mu)==false &&
         norm(rb) <= norm(rb0)*beta*mu/mu_0 && norm(rc) <= norm(rc0)*beta*mu/mu_0
end

function UpdateAlphaOpt(gamma, beta, A, b, x, lambda, s, dir_k, mu, x_0, lambda_0, s_0, mu_0, alpha_step)
    alpha = 0
    x_k, lambda_k, s_k = x, lambda, s
    n = length(x)
    m = length(lambda)
    mu_k = s_k' * x_k / n
    while Point_in_Neighborhood_infty(gamma, beta,  A, b, x_k,lambda_k, s_k, mu_k, x_0, lambda_0, s_0, mu_0) == true 
        alpha += alpha_step
        x_k = x + alpha*dir_k[1:n]
        lambda_k = lambda + alpha*dir_k[n+1: n+m]
        s_k = s + alpha*dir_k[n+m+1: end]
        if s_k' * x_k / n <= (1 - 0.01*alpha) * mu_k # armijo
            mu_k = s_k' * x_k / n
        else
            break
        end
    end
    largest_alpha = alpha-alpha_step
    x_k = x + largest_alpha*dir_k[1:n]
    lambda_k = lambda + largest_alpha*dir_k[n+1: n+m]
    s_k = s + largest_alpha*dir_k[n+m+1: end]
    
    return x_k, lambda_k, s_k        
end

UpdateAlphaOpt (generic function with 2 methods)

In [180]:

function InfeasiblePathFollowing1(c, A, b, init_vect, gamma, beta, sigma_min, sigma_max, 
    max_iter, tol_step_x= 1e-4, eps = 1e-4, alpha_step=1e-3)

    x_0, lambda_0, s_0 = init_vect
    x_k, lambda_k, s_k = copy(x_0), copy(lambda_0), copy(s_0)
    n_rows, n_cols = size(A) 
    rb_0 = A * x_0 - b
    rc_0 = A' * lambda_0 + s_0 - c
    mu_0 = s_0' * x_0 / n_cols
    k = 0
    step_x = 1
    pdd = 1
    e = ones(n_cols, 1)
    while k <= max_iter && step_x > tol_step_x && pdd > eps
        Xk = Diagonal(x_k)
        Sk = Diagonal(s_k)
        x_k_prec = copy(x_k)
        mu_k = s_k' * x_k / n_cols
        rb_k = A * x_k - b
        rc_k = A' * lambda_k + s_k - c
        sigma_k = sigma_min  # comment choisir sigma en pratique? ici je prends toujours sigma_min
        
        Jacob_Fk = [zeros(n_cols, n_cols) A' I(n_cols)
                    A zeros(n_rows, n_rows) zeros(n_rows, n_cols)
                    Sk zeros(n_cols, n_rows) Xk]
        Fk = [-rc_k
                -rb_k
                -Xk*Sk*e + sigma_k*mu_k*e]

        dir_k = Jacob_Fk\Fk

        x_k, lambda_k, s_k = UpdateAlphaOpt(gamma, beta, A, b, x_k, lambda_k, s_k, dir_k,
                                            mu_k, x_0, lambda_0, s_0, mu_0, alpha_step)
        
        step_x = norm(x_k - x_k_prec)
        pdd = primal_dual_diff(c, b, x_k, lambda_k)
        k += 1
        
        # affichage
        println("---------------------------------------------------------------")
        println("Iter ", k, ":  step_x = ", step_x, "   pdd = ", pdd)
    end
    
    criteria = [k > max_iter, step_x <= tol_step_x, pdd <= eps]
    criteria_names = ["reached max_iter", "step_x <= tol_step_x", "pdd <= eps"]
    println("\n stopping criterion = ",criteria_names[findall(criteria)])
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, 
        "n_iter" => k, "pdd" => pdd, "step_x" => step_x)
end

InfeasiblePathFollowing1 (generic function with 6 methods)

In [181]:
### probleme 1
c = [1, 0 ,0]
A = Matrix([1, 1, 1]')
b = [1];

In [182]:
# initialisation 1 
x01 = [10, 1, 3]
lambda01 = [-2]
s01 = [3, 4, 2]
init_vect1 = (x01, lambda01, s01);
gamma, beta = 0.1, 2  # pour ce problème, ne marche pas lorque gamma est trop grand.
res_ipf1 =  InfeasiblePathFollowing1(c, A, b, init_vect1, gamma, beta, 0.2, 0.3, 100);
display_results(res_ipf1) 

---------------------------------------------------------------
Iter 1:  step_x = 0.9734740141142717   pdd = 1.1036673492491411
---------------------------------------------------------------
Iter 2:  step_x = 0.8685952321447789   pdd = 1.1176466146862227
---------------------------------------------------------------
Iter 3:  step_x = 0.7845149477312834   pdd = 1.1330469114218886
---------------------------------------------------------------
Iter 4:  step_x = 0.7083750827877169   pdd = 1.1499087856355672
---------------------------------------------------------------
Iter 5:  step_x = 0.6394854030062069   pdd = 1.168249714641063
---------------------------------------------------------------
Iter 6:  step_x = 0.5772114309281322   pdd = 1.1880595005965155
---------------------------------------------------------------
Iter 7:  step_x = 0.5150474968641687   pdd = 1.2090347978179206
---------------------------------------------------------------
Iter 8:  step_x = 0.465438331524679   pdd

---------------------------------------------------------------
Iter 82:  step_x = 0.004317761783280066   pdd = 0.12246353858357115
---------------------------------------------------------------
Iter 83:  step_x = 0.004053465716646481   pdd = 0.11458633341923284
---------------------------------------------------------------
Iter 84:  step_x = 0.003802856274458719   pdd = 0.10718935384197756
---------------------------------------------------------------
Iter 85:  step_x = 0.0035655990194084046   pdd = 0.10024651164678745
---------------------------------------------------------------
Iter 86:  step_x = 0.0033413092852359683   pdd = 0.09373276279918476
---------------------------------------------------------------
Iter 87:  step_x = 0.0031295632752433764   pdd = 0.08762410878506982
---------------------------------------------------------------
Iter 88:  step_x = 0.002929907859796796   pdd = 0.08189759223982691
---------------------------------------------------------------
Iter 89: 

OrderedDict{String,Any} with 6 entries:
  "x_opt"      => [0.0136177, 0.484114, 0.503973]
  "lambda_opt" => [-0.0204107]
  "s_opt"      => [1.02041, 0.020673, 0.0204107]
  "n_iter"     => 101
  "pdd"        => 0.0335712
  "step_x"     => 0.00121548